#### Now I try to play with the pixel to pixel radial gradient of Balmer decrement


In [1]:
from    astropy.table       import Table
from    scripts.tools       import *
import  numpy               as     np
from    astropy.io          import fits
from    astropy.cosmology   import Planck18
import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
import matplotlib
#matplotlib.use('Agg')
from    functools           import reduce
from    tqdm                import tqdm
import  os
import  gc                                         
import warnings
warnings.filterwarnings("ignore")
# Avoid division by zero
np.errstate(divide='ignore', invalid='ignore')
warnings.filterwarnings("ignore", message="marker is redundantly defined")


#calculate distance of each pixel to center:
def image_to_center_distance(pixel_length):
    y,x = np.indices((50,50))
    center = (np.array((50,50)) - 1) / 2
    distance = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    return (distance*pixel_length).value

#this is the joint function that:
#1. calculate the 2d map of balmer decrement + error
#calculate each pixel distance using redshift + return a table of balmer decrement vs distance
def extract_radial_profile(obj,
                            LINE_HA='LINE_HA',LINE_HB='LINE_HB',
                            LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB'):
    
    #load corresponding extracted file
    path = f"data_extracted/{file_name(obj,'extracted')}"
    with fits.open(path,mode='update') as extracted:

        #seg ha hb from extracted data
        seg = find_data('SEG',extracted)[1].data == obj['ID']
        ha  = find_data(LINE_HA,extracted)[1]
        hb  = find_data(LINE_HB,extracted)[1]
        #this mask out all the the negative pixel due to continium oversubtraction
        mask1 = np.logical_and(find_data('LINE_HA',extracted)[1].data>0, find_data('LINE_HB',extracted)[1].data>0)
        mask2 = np.logical_and(ha.data>0,hb.data>0)
        mask  = np.logical_and(np.logical_and(mask1,mask2),seg)

        #pixel length from redshift, !! here angular diameter distance is used
        #to  center distance
        pixel_length = np.deg2rad(ha.header['PIXASEC']/3600) * Planck18.angular_diameter_distance(obj['z_MAP']).to(u.kpc)
        distances_obj  = image_to_center_distance(pixel_length)[mask]
        sort = np.argsort(distances_obj)
        #this gives the distance sorted out.
        distances_obj = distances_obj[sort]
        
        #this part calculate the actual 2d balmer decrement + error
        balmer_2d     = fits.ImageHDU(data = ha.data/hb.data,name = '2D_BALMER')
        #!!!! note the error here is just using the square root of the image thumbnail, weightmap needed for improvement!!
        balmer_2d_err = fits.ImageHDU(data = ha.data/hb.data**2 + ha.data**2 * hb.data/hb.data**2,name = '2D_BALMER_ERR')

        balmer_pix     = balmer_2d.data[mask][sort]
        balmer_pix_err = balmer_2d_err.data[mask][sort]
        dtype = [('DISTANCE [kpc]', 'f4'), ('BALMER_DECREM_PIX', 'f4'), ('BALMER_DECREM_PIX_ERR', 'f4')]

        #choose the right name for saving the radial table
        if 'CONV' in LINE_HB:
            name_addon = '_CONV'
        else:
            name_addon = ''
        if 'BG' not in LINE_HA:
            name = f'PIX_RAD_PROFILE{name_addon}'
        else:
            name = f'PIX_RAD_PROFILE{name_addon}_BG'

        balmer_table = fits.BinTableHDU(np.array(list(zip(distances_obj, balmer_pix, balmer_pix_err)), dtype=dtype), name=name)

        #save update
        save_update(balmer_2d,extracted)
        save_update(balmer_2d_err,extracted)
        save_update(balmer_table,extracted)
        extracted.flush()
    return f"{obj['subfield']}-{obj['ID']} processed"


from    concurrent.futures  import ThreadPoolExecutor, as_completed
def cat_process(obj_lis,
                LINE_HA,   LINE_HB,
                LINEWHT_HA, LINEWHT_HB,
                max_threads=1):
        print(f'start process,{LINE_HA},{LINE_HB}')
        results = []
        if max_threads > 1 :
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(
                    extract_radial_profile,
                    obj,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB
                                            ): obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in tqdm(obj_lis):
                results.append(extract_radial_profile(obj,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB))
            return results

def main():
    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process(obj_lis,
                          LINE_HA='LINE_HA',LINE_HB='LINE_HB',
                          LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB',max_threads=3)
    errorcounting(results)

    results = cat_process(obj_lis,
                          LINE_HA='LINE_HA',LINE_HB='LINE_HB_CONV',
                          LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV',max_threads=3)
    errorcounting(results)

if __name__ == '__main__' :
    main()

start process,LINE_HA,LINE_HB


Processing: 100%|██████████| 158/158 [02:33<00:00,  1.03it/s]


total number of obj processed: 158
number of failed obj 0
start process,LINE_HA,LINE_HB_CONV


Processing: 100%|██████████| 158/158 [00:48<00:00,  3.27it/s]

total number of obj processed: 158
number of failed obj 0
